# **National Id**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
from PIL import Image
import csv
import h5py
import numpy as np
import random 


##### Bank Card requirements

pattern = np.array([2, 1])
pattern = np.tile(pattern, 7)
pattern = np.append(pattern, 2)
# print(pattern)

bank_type = [603799, 589210, 627648, 627961, 603770, 628023, 627760, 502908,
             627412, 622106, 502229, 629599, 627488, 621986, 639346, 639607, 
             504706, 502806, 502938, 603769, 610433, 627353, 585983, 589463, 
             627381, 639370, 507677, 628157, 505801, 606256, 606373]

indices = np.arange(10, 1, -1)             
# print(indices)

def num_to_list(n):
  return [int(x) for x in str(n)]        


def bank_card_control_digit(num):
  mul = np.multiply(pattern, num)
  mul = [d-9 if d>9 else d for d in mul]
  sum = np.sum(mul)
  control = 10 - (sum % 10)
  if(control == 10):
      return 0
  return control

def bank_num_generator():
  idx = random.randint(0, len(bank_type) -1)
  # append first 6 digits to card numbers
  num = num_to_list(bank_type[idx])
  # generate 9 next random digits
  for i in range(9):
    num.append(random.randint(0, 9))
  # calculate control digit
  control = bank_card_control_digit(num)
  num.append(control)
  return ''.join(map(str, num))  


##### National ID requirements

def id_card_control_digit(num):
  mul = np.multiply(indices, num)
  sum = np.sum(mul) % 11
  control = 0
  if sum < 2:
    control = sum
  else: control = 11 - sum
  return control

def id_num_generator():
  # generate 9 first random digits
  num = []
  for i in range(9):
    num.append(random.randint(0, 9))
    
  # calculate control digit
  control = id_card_control_digit(num)
  num.append(control)
  return ''.join(map(str, num))  

def convert_to_farsi(id):
    s = list(id)
    for i in range(len(s)):
        if(s[i] == "0"):
            s[i] = "۰"
        elif (s[i] == "1"):
            s[i] = "۱"
        elif (s[i] == "2"):
            s[i] = "۲"
        elif (s[i] == "3"):
            s[i] = "۳"
        elif (s[i] == "4"):
            s[i] = "۴"
        elif (s[i] == "5"):
            s[i] = "۵"
        elif (s[i] == "6"):
            s[i] = "۶"
        elif (s[i] == "7"):
            s[i] = "۷"
        elif (s[i] == "8"):
            s[i] = "۸"
        elif (s[i] == "9"):
            s[i] = "۹"
        
    return "".join(s)

In [ ]:
import pandas as pd  
 
path = Path("/content/gdrive/My Drive/newData3/")
path.mkdir(parents=True, exist_ok=True)


dict = {'images': [], 'labels': []}  
df = pd.DataFrame(dict) 
df.to_csv(path / 'data.csv') 

In [ ]:
import pandas as pd  
from csv import DictWriter

field_names = ['','images','labels']
  
def store_png(images, labels, start):
    with open(path / 'data.csv', 'a') as f_object:
        dictwriter_object = DictWriter(f_object, fieldnames=field_names)

        for i in range(len(images)):
            image_name =(str(i + start) + ".png")
            dict = {'' : str(i+start) ,'images': image_name, 'labels': labels[i]}  
            dictwriter_object.writerow(dict)
            images[i].save(path / image_name)
        f_object.close()

    
    # dict = {'images': images_path, 'labels': labels}  
    # df = pd.DataFrame(dict) 
    # df.to_csv(path / 'data.csv') 


In [ ]:
Images = [] 
Labels = []


for i in range(2500):
    # image, cardNumber = generateBankCardData()
    # image, cardNumber = generateOldNationalIdData()
    image, cardNumber = generateNewNationalIdData()
    Images.append(image)
    Labels.append(cardNumber)


In [ ]:
start = 0

In [ ]:
store_png(Images, Labels, start)
start += len(Images)

In [ ]:
start

10000

In [ ]:
def generateNewNationalIdData():
    # NationalId = "0022939776"
    NationalId = id_num_generator()
    NationalIdFa = convert_to_farsi(NationalId)

    img = Image.open("background_hooshmand_barik.jpg")
    d = ImageDraw.Draw(img)
    font = ImageFont.truetype("Yekan.ttf", size=35)
    d.text((50,15), NationalId , fill=(0,0,0), font=font) 
    return img, NationalIdFa

def generateOldNationalIdData():
    # NationalId = "۰۹۳-۹۰۹۲۰۰-۱"
    NationalId = id_num_generator()
    NationalIdFa = convert_to_farsi(NationalId)
    BankNumberWithSpace = NationalIdFa[0:3] + "-" + NationalIdFa[3:9] + "-" + NationalIdFa[-1]
    img = Image.open("background_ghadimi_asli.jpg")
    d = ImageDraw.Draw(img)
    font = ImageFont.truetype("yas.ttf", size=40)
    d.text((35,0), BankNumberWithSpace, fill=(0,0,0), font=font)
    return img, NationalIdFa


# Unused

In [ ]:

hdf5_dir_nationalId = Path("/content/gdrive/My Drive/data/NationalId/")
hdf5_dir_nationalId.mkdir(parents=True, exist_ok=True)


def store_new_nationalIdData(images_path, labels):
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir_nationalId / f"NewNationalIdData_{num_images}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.string_dtype(encoding='utf-8'), data=labels
    )
    file.close()


def store_old_nationalIdData(images, labels):
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir_nationalId / f"OldNationalIdData_{num_images}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.string_dtype(encoding='utf-8'), data=labels
    )
    file.close()


def read_NationalIdData(num_images):
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir_nationalId / f"NationalIdData_{num_images}.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/meta"]).astype("str")

    return images, labels

In [ ]:
OldNationalIdImages = [] 
OldNationalIdLabels = []

NewNationalIdImages = [] 
NewNationalIdLabels = []

for i in range(50000):
    image, cardNumber = generateNewNationalIdData()
    NewNationalIdImages.append(image)
    NewNationalIdLabels.append(cardNumber)
    image, cardNumber = generateOldNationalIdData()
    OldNationalIdImages.append(image)
    OldNationalIdLabels.append(cardNumber)
    

In [ ]:
store_new_nationalIdData(NewNationalIdImages, NewNationalIdLabels)

In [ ]:
store_old_nationalIdData(OldNationalIdImages, OldNationalIdLabels)

In [ ]:
readedImgN, readedLabelsN = read_NationalIdData(100000) 
display(Image.fromarray(readedImgN[100], 'RGB'))
print(readedLabelsN[100])

# Bank Card 

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def generateBankCardData():
    BankNumber = bank_num_generator()
    BankNumberWithSpace = BankNumber[0:4] + "  " + BankNumber[4:8] + "  " + BankNumber[8:12] + "  " + BankNumber[12:16];

    fonts = ["calibri.ttf", "arial.ttf", "bahnschrift.ttf", "calibrib.ttf", "comic.ttf",
            "framd.ttf", "GOTHIC.TTF", "HARLOWSI.TTF", "micross.ttf", "segoeui.ttf"]
    fontSize = [26, 23, 24, 25, 21,
                22, 23, 26, 23, 23]

    backGrounds = ["background1.jpg", "background2.jpg", "background3.jpg",
                "background4.jpg", "background5.jpg", "background6.jpg",
                "background7.jpg", "background8.jpg", "background9.jpg", "background10.jpg"]
    FontColor = [(0,0,0), (255,255,255), (0,0,0),
                (230,184,0), (0,0,0), (0,0,0),
                (0,0,0), (0,0,0), (0,0,0), (0,0,0)]

    FontRandomIdx = random.randint(0, len(fonts) -1)
    BackgroundRandomIdx = random.randint(0, len(backGrounds) -1)

    # print("font:",fonts[FontRandomIdx])
    # print("font size:",fontSize[FontRandomIdx])
    # print("background:",backGrounds[BackgroundRandomIdx])
    # print("font color",FontColor[BackgroundRandomIdx])


    img = Image.open(backGrounds[BackgroundRandomIdx])
    d = ImageDraw.Draw(img)

    font = ImageFont.truetype(fonts[FontRandomIdx], size=fontSize[FontRandomIdx])
    d.text((5,4), BankNumberWithSpace, fill=FontColor[BackgroundRandomIdx], font=font)
    # display(img)
    # print("label: ", BankNumber)
    return img, str(BankNumber)

# Unused bank

In [ ]:
hdf5_dir = Path("/content/gdrive/My Drive/data/KarteBanki/")
hdf5_dir.mkdir(parents=True, exist_ok=True)


def store(images, labels):
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"BankCardData_{num_images}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.string_dtype(encoding='utf-8'), data=labels
    )
    file.close()


def Read(num_images):
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"BankCardData_{num_images}.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/meta"]).astype("str")

    return images, labels

In [ ]:
Images = [] 
Labels = []

for i in range(100000):
    image, cardNumber = generateBankCardData()
    Images.append(image)
    Labels.append(cardNumber)


In [ ]:
store(Images, Labels)

In [ ]:
readedImg, readedLabels = Read(100000) 
display(Image.fromarray(readedImg[99999], 'RGB'))
print(readedLabels[99999])

OSError: ignored